## Introduction and Schema Diagram

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

In [2]:
%%sql
SELECT 
     name
    ,type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql

SELECT * FROM customer LIMIT 10;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
7,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5
8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4
9,Kara,Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,None,kara.nielsen@jubii.dk,4
10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4


## Selecting New Albums to Purchase

In [4]:
%%sql
WITH 
    customer_info AS
                (SELECT customer_id, country FROM customer WHERE country = 'USA'),
    track_info AS 
                (SELECT c.customer_id
                      , c.country
                      , i.invoice_id
                      , li.track_id
                      , t.name AS track_name
                      , t.genre_id
                 FROM customer_info c
                 LEFT JOIN invoice i ON i.customer_id = c.customer_id 
                 LEFT JOIN invoice_line li ON li.invoice_id = i.invoice_id
                 LEFT JOIN track t ON t.track_id = li.track_id),
    genre_info AS
               (SELECT t.* ,
                       g.name AS genre
                FROM track_info t
                LEFT JOIN genre g ON g.genre_id = t.genre_id)
        

        
        
SELECT genre AS Genre
     , COUNT(track_id) AS tracks_sold
     , CAST(COUNT(track_id) AS FLOAT) / (SELECT COUNT(*) FROM genre_info) AS percentage
FROM genre_info
GROUP BY genre 

Done.


Genre,tracks_sold,percentage
Alternative,35,0.03330161750713606
Alternative & Punk,130,0.12369172216936251
Blues,36,0.03425309229305423
Classical,4,0.003805899143672693
Easy Listening,13,0.012369172216936251
Electronica/Dance,5,0.004757373929590866
Heavy Metal,3,0.0028544243577545195
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425
Latin,22,0.02093244529019981


The three music genre's I recommend for purchase to be sold at this store is Rock, Metal and Alternative and Punk.

## Analyzing Employee Sales Performance

In [5]:
%%sql

WITH
    customer_info AS
            (SELECT c.customer_id, c.support_rep_id, i.invoice_id, SUM(i.total) total_purchase
             FROM customer c
             LEFT JOIN invoice i ON i.customer_id = c.customer_id
             GROUP BY c.customer_id
             ,c.support_rep_id
             ,i.invoice_id),
    rep_totals AS
           (SELECT support_rep_id, SUM(total_purchase) AS total
               FROM customer_info
               GROUP BY support_rep_id)

SELECT  e.employee_id
       , e.first_name || ' ' || e.last_name AS employee_name
       , r.total
FROM employee e
LEFT JOIN rep_totals r ON r.support_rep_id = e.employee_id

Done.


employee_id,employee_name,total
1,Andrew Adams,None
2,Nancy Edwards,None
3,Jane Peacock,1731.510000000004
4,Margaret Park,1584.0000000000032
5,Steve Johnson,1393.9200000000028
6,Michael Mitchell,None
7,Robert King,None
8,Laura Callahan,None


In [6]:
%%sql

SELECT * FROM customer
WHERE support_rep_id = 8

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id


## Analyzing Sales by Country

In [7]:
%%sql
WITH 
    customer_info AS
                (SELECT customer_id, country FROM customer),
    track_info AS 
                (SELECT c.customer_id
                      , c.country
                      , i.invoice_id
                      , li.track_id
                      , t.name AS track_name
                      , t.genre_id
                      , i.total
                 FROM customer_info c
                 LEFT JOIN invoice i ON i.customer_id = c.customer_id 
                 LEFT JOIN invoice_line li ON li.invoice_id = i.invoice_id
                 LEFT JOIN track t ON t.track_id = li.track_id),
    country_info AS (SELECT country
                    ,COUNT(DISTINCT customer_id) AS number_of_customers
                    ,SUM(total) AS total_sales
                    ,AVG(total) AS avg_sales
                    ,CASE WHEN COUNT(DISTINCT customer_id) = 1 THEN 1
                          ELSE 0
                         END AS sort
                     FROM track_info 
                     GROUP BY country
                      ORDER BY total_sales),
    other_info AS 
               (
SELECT CASE 
           WHEN sort = 1 THEN 'Other'
           ELSE country
           END AS country
    ,number_of_customers
    ,total_sales
    ,avg_sales
    ,sort
FROM country_info)   
        
        
SELECT country,
      SUM(number_of_customers) AS number_of_customers
    , SUM(total_sales) AS total_sales
    , AVG(avg_sales) AS avg_sales
    , sort 
FROM other_info
GROUP BY country, sort
ORDER BY sort ASC, total_sales DESC 


Done.


country,number_of_customers,total_sales,avg_sales,sort
USA,13,10405.889999999954,9.900941960038015,0
Canada,8,5489.5500000000075,10.14704251386323,0
Brazil,5,4058.9999999999977,9.395833333333329,0
France,5,3972.8700000000053,10.109083969465663,0
Germany,4,3441.239999999997,10.181183431952654,0
Czech Republic,2,3183.839999999992,11.535652173913014,0
United Kingdom,3,2498.759999999999,10.075645161290318,0
India,2,1887.9300000000064,10.205027027027063,0
Portugal,2,1755.2700000000068,9.38647058823533,0
Other,15,10808.819999999996,9.476255196854986,1


Based on the data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

In [8]:
%%sql

SELECT * FROM customer WHERE country = 'USA' LIMIT 5

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4


In [9]:
%%sql
WITH 
    customer_info AS
                (SELECT customer_id, country FROM customer WHERE country = 'USA'),
    track_info AS 
                (SELECT c.customer_id
                      , c.country
                      , i.invoice_id
                      , li.track_id
                      , t.name AS track_name
                      , t.genre_id
                 ,i.total
                 FROM customer_info c
                 LEFT JOIN invoice i ON i.customer_id = c.customer_id 
                 LEFT JOIN invoice_line li ON li.invoice_id = i.invoice_id
                 LEFT JOIN track t ON t.track_id = li.track_id)
        

        
        
SELECT COUNT(DISTINCT customer_id) 
       ,SUM(total)
       ,AVG(total)
FROM track_info
GROUP BY country

Done.


COUNT(DISTINCT customer_id),SUM(total),AVG(total)
13,10405.889999999954,9.900941960038015


In [10]:
%%sql

SELECT i.invoice_id
     , i.customer_id
    ,t.track_id
    ,t.name AS track_name
    ,t.album_id  
    ,a.title AS album_name
FROM invoice i 
LEFT JOIN invoice_line li ON li.invoice_id = i.invoice_id
LEFT JOIN track t ON t.track_id = li.track_id
LEFT JOIN album a ON a.album_id = t.album_id
LIMIT 10

Done.


invoice_id,customer_id,track_id,track_name,album_id,album_name
16,1,2160,Nothingman,178,Live On Two Legs [live]
16,1,875,Acelerou,70,Djavan Ao Vivo - Vol. 1
16,1,83,Wherever I May Roam,9,Plays Metallica By Four Cellos
16,1,16,Dog Eat Dog,4,Let There Be Rock
16,1,1756,Magamalabares,145,Barulhinho Bom
16,1,3460,Love Is a Losing Game,321,Back to Black
16,1,93,Exploder,10,Audioslave
16,1,24,Love In An Elevator,5,Big Ones
16,1,21,Hell Ain't A Bad Place To Be,4,Let There Be Rock
77,1,15,Go Down,4,Let There Be Rock


## Albums vs Individual Tracks

In [11]:
%%sql

with all_tables as (
        SELECT *
        FROM invoice as i
        LEFT JOIN invoice_line as il on il.invoice_id = i.invoice_id
        LEFT JOIN track as tr on tr.track_id = il.track_id
    ),
    invoice_albums as (
        SELECT invoice_id,
               album_id as album_ids,
               track_id as track_ids
        FROM all_tables
        GROUP BY 1,2,3
    ),
    albums as (
        SELECT  album_id,
                track_id
        FROM track
        GROUP BY 1,2
    ),
    album_purchase as (
        SELECT
            ia.invoice_id,
            CASE
                WHEN COUNT(distinct ia.album_ids) = 1
                AND COUNT(distinct ia.track_ids) = COUNT(DISTINCT a.track_id)
                THEN 'album_purchase'
                ELSE 'non-album_purchase'
            END as album_purchase
        FROM invoice_albums as ia
        LEFT JOIN albums as a ON ia.album_ids = a.album_id
        GROUP BY 1        
    )

SELECT
    album_purchase,
    COUNT(invoice_id) as invoices,
    CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM album_purchase) * 100
FROM album_purchase
GROUP BY 1

Done.


album_purchase,invoices,CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM album_purchase) * 100
album_purchase,114,18.566775244299674
non-album_purchase,500,81.43322475570032


### Few More Ideas to Explore

* Which artist is used in the most playlists?
* How many tracks have been purchased vs not purchased?
* Is the range of tracks in the store reflective of their sales popularity?
* Do protected vs non-protected media types have an effect on popularity?